## 1. Chatbot slot

In [1]:
input_data = '판교에 지금 주문해줘'
output_data = ''

request = {
    'intent_id':'',
    'input_data':input_data,
    'request_type':'text',
    'story_slot_entity':{},
    'output_data':output_data
}

intent_list = {
    '주문':['주문', '배달'],
    '예약':['예약', '잡아줘'],
    '정보':['정보', '알려']
}

story_slot_entity = {
    '주문':{
        '메뉴':None,
        '장소':None,
        '날짜':None
    },
    '예약':{
        '장소':None,
        '날짜':None
    },
    '정보':{
        '대상':None
    }
}

In [4]:
from konlpy.tag import Mecab
mecab = Mecab()

In [5]:
preprocessed = mecab.pos(request.get('input_data'))
print(preprocessed)

[('판교', 'NNG'), ('에', 'JKB'), ('지금', 'MAG'), ('주문', 'NNG'), ('해', 'XSV+EC'), ('줘', 'VX+EC')]


In [6]:
intent_id = '주문'
slot_value = story_slot_entity.get('주문')

In [7]:
menu_list = ['피자', '햄버거', '치킨']
loc_list = ['판교', '야탑', '서현']
date_list = ['지금', '내일', '모레']

In [8]:
for pos_tag in preprocessed:
    if pos_tag[1] in ['NNG', 'NNP', 'SL', 'MAG']:
        if pos_tag[0] in menu_list:
            slot_value['메뉴'] = pos_tag[0]
        elif pos_tag[0] in loc_list:
            slot_value['장소'] = pos_tag[0]
        elif pos_tag[0] in date_list:
            slot_value['날짜'] = pos_tag[0]

print(story_slot_entity.get('주문'))

{'메뉴': None, '장소': '판교', '날짜': '지금'}


In [9]:
if (None in slot_value.values()):
    key_values = ""
    for key in slot_value.keys():
        if (slot_value[key] is None):
            key_values = key_values+key+""
        output_data = key_values+' 선택해 주세요'
else:
    output_data = '주문이 완료되었습니다.'
print(output_data)

메뉴 선택해 주세요


In [10]:
response = {
    'intent_id' : '',
    'input_data' : input_data,
    'request_type' : 'text',
    'story_slot_entity' : {},
    'output_data' : ''
}
response['output_data'] = output_data
print(response["output_data"])

메뉴 선택해 주세요


## 2. Word Representation

In [11]:
one_data = [
    ['안녕', '만나서 반가워'],
    ['넌 누구니', '나는 AI로봇이란다.'],
    ['피자 주문 할게', '음료도 주문해줘'],
    ['음료는 뭘로','콜라로 해줘']
]

In [12]:
train_data = list(map(lambda x : mecab.morphs(''.join(x)), one_data))

In [13]:
import itertools
train_data = list(itertools.chain.from_iterable(train_data))

In [14]:
print(list(train_data))

['안녕', '만나', '서', '반가워', '넌', '누구', '니', '나', '는', 'AI', '로', '봇', '이', '란다', '.', '피자', '주문', '할', '게', '음료', '도', '주문', '해', '줘', '음료', '는', '뭘로', '콜라', '로', '해', '줘']


In [15]:
import numpy as np
bucket = np.zeros(len(train_data), dtype=np.float)

In [16]:
for word in train_data:
    bucket_temp = bucket.copy()
    np.out(bucket_temp, train_data.index(word), 1) # out이 없음
    print(bucket_temp)

AttributeError: module 'numpy' has no attribute 'out'

In [17]:
# Word2Vec
from gensim.models import Word2Vec

In [18]:
train_data = [train_data]
print(train_data)

[['안녕', '만나', '서', '반가워', '넌', '누구', '니', '나', '는', 'AI', '로', '봇', '이', '란다', '.', '피자', '주문', '할', '게', '음료', '도', '주문', '해', '줘', '음료', '는', '뭘로', '콜라', '로', '해', '줘']]


In [19]:
model = Word2Vec(train_data,size=50, window=2, min_count=1)
#model.build_vocab(train_data)
#model.train(train_data)
print('model check : {0}'.format(model))

model check : Word2Vec(vocab=25, size=50, alpha=0.025)


## Intent와 NER 모델을 만들기 위한 Data의 구성 방법  
피자주문 / 숙소예약 / 여행정보의 각각의 Entity 구성

In [20]:
train_data_order = ['판교에 오늘 피자 주문해줘']
train_data_reserve = ['오늘 날짜에 호텔 예약 해줄래']
train_data_info = ['모레 날짜의 판교 여행 정보 알려줘']

get_data_list = train_data_info[0]

dict_entity = {
    'date' : ['오늘', '내일', '모레'],
    'loc' : ['판교', '야탑'],
    'menu' : ['피자', '햄버거'],
    'hotel' : ['호텔', '여관', '민박'],
    'travel' : ['여행', '관광', '카페']
}

In [21]:
length = 1
for key in list(dict_entity.keys()):
    length = length * len(dict_entity[key])
print("Augmentation length is {0}".format(length))

Augmentation length is 108


In [22]:
morphed_text = mecab.pos(get_data_list)
print(morphed_text)

[('모레', 'MAG'), ('날짜', 'NNG'), ('의', 'JKG'), ('판교', 'NNG'), ('여행', 'NNG'), ('정보', 'NNG'), ('알려줘', 'VV+EC+VX+EC')]


In [23]:
tagged_text = ''
for pos_tag in morphed_text:
    if(pos_tag[1] in ['NNG', 'MAG', 'NNP', 'SL'] and len(pos_tag[0]) > 1):
        feature_value = pos_tag[0]
        tagged_text = tagged_text + pos_tag[0] + ' '
print(tagged_text)

모레 날짜 판교 여행 정보 


In [24]:
pattern = ''
for word in tagged_text.split(' '):
    entity = list(filter(lambda key : word in dict_entity[key], list(dict_entity.keys())))
    if (len(entity) > 0):
        pattern = pattern + 'tag' + entity[0] + ' '
    else:
        pattern = pattern + word + ' '
print(pattern)

tagdate 날짜 tagloc tagtravel 정보  


## Data augmentation

In [25]:
def augmentation_pattern(pattern, dict_entity):
    # 입력된 패턴을 List로 바꿈
    aug_pattern = pattern.split(' ')
    # Augment된 Text List
    augmented_text_list = []
    # copy를 위한 임시 list
    temp_aug = []
    for i in range(0, len(aug_pattern)):
        # Entity에 해당하는 값일 경우 Entity List를 가져옴
        if aug_pattern[i].find('tag') > -1 : # tag가 있으면 0, 없으면 1
            dict_list = dict_entity[aug_pattern[i].replace('tag',"")]
            # 각 Entity 별로 값을 append하면서 pattern 구성
            for j in range(0, len(dict_list)):
                # 최초 Entity값은 그냥 추가만
                if i == 0:
                    augmented_text_list.append(dict_list[j] + " ")
                elif j == 1:
                    augmented_text_list = list(filter(lambda word : len(word.split(' ')) == i+1, augmented_text_list))
                    copy_data_order = augmented_text_list * (len(dict_list)-2)
                    augmented_text_list = list(map(lambda x : x + dict_list[j] + " ", augmented_text_list))
                    augmented_text_list = augmented_text_list + temp_aug + copy_data_order
                else:
                    # list의 수를 체크하여 값을 추가
                    temp_aug = list(filter(lambda word : len(word.split(' ')) == i+1, augmented_text_list))
                    temp_aug = list(map(lambda x : x + dict_list[j] + " ", temp_aug))
                    # 추가된 list를 위해 기존 값 삭제
                    if j != 0:
                        augmented_text_list = augmented_text_list[0:len(augmented_text_list) - len(temp_aug)]
                    augmented_text_list = augmented_text_list + temp_aug
        # Entity 추가 대상이 아닐 경우 Pattern만 추가
        else:
            augmented_text_list = list(map(lambda x : x + aug_pattern[i] + " ", augmented_text_list))
        # N*N으로 증가시키기 위한 list
        temp_aug = augmented_text_list
    return augmented_text_list

In [26]:
augmented_text_list = augmentation_pattern(pattern, dict_entity)
augmented_text_list

['오늘 날짜 야탑 관광 정보   ',
 '내일 날짜 야탑 관광 정보   ',
 '모레 날짜 야탑 관광 정보   ',
 '오늘 날짜 판교 관광 정보   ',
 '내일 날짜 판교 관광 정보   ',
 '모레 날짜 판교 관광 정보   ',
 '오늘 날짜 야탑 여행 정보   ',
 '내일 날짜 야탑 여행 정보   ',
 '모레 날짜 야탑 여행 정보   ',
 '오늘 날짜 판교 여행 정보   ',
 '내일 날짜 판교 여행 정보   ',
 '모레 날짜 판교 여행 정보   ',
 '오늘 날짜 야탑 카페 정보   ',
 '내일 날짜 야탑 카페 정보   ',
 '모레 날짜 야탑 카페 정보   ',
 '오늘 날짜 판교 카페 정보   ',
 '내일 날짜 판교 카페 정보   ',
 '모레 날짜 판교 카페 정보   ']

## Bio Tagging

In [27]:
def augmentation_bio_pattern(pattern, dict_entity):
    # 입력된 패턴을 list로 바꿈
    aug_pattern = pattern.split(' ')
    # augment된 text list
    augmented_text_list = []
    # copy를 위한 임시 list
    temp_aug = []
    for i in range(0, len(aug_pattern)):
        # Entity에 해당하는 값일 경우 Entity List를 가져옴
        if aug_pattern[i].find('tag') > -1:
            dict_list = dict_entity[aug_pattern[i].replace('tag', "")]
            bio_tag = aug_pattern[i].replace('tag', "B_")
            # 각 Entity별로 값을 append하면서 pattern 구성
            for j in range(0, len(dict_list)):
                # 최초 entity 값은 추가만
                if i == 0:
                    augmented_text_list.append(bio_tag+" ")
                elif j == 1:
                    augmented_text_list = list(filter(lambda word : len(word.split(' ')) == i+1, augmented_text_list))
                    copy_data_order = augmented_text_list * (len(dict_list)-2)
                    augmented_text_list = list(map(lambda x : x + bio_tag + " ", augmented_text_list))
                    augmented_text_list = augmented_text_list + temp_aug + copy_data_order
                else:
                    # list의 수를 체크하여 값 추가
                    temp_aug = list(filter(lambda word : len(word.split(' ')) == i+1, augmented_text_list))
                    temp_aug = list(map(lambda x : x + bio_tag + " ", temp_aug))
                    # 추가된 list를 위해 기존 값 삭제
                    if j != 0:
                        augmented_text_list = augmented_text_list[0:len(augmented_text_list) - len(temp_aug)]
                        augmented_text_list = augmented_text_list + temp_aug
        # Entity 추가 대상이 아닐 경우 pattern만 추가
        else:
            augmented_text_list = list(map(lambda x : x + aug_pattern[i] + " ", augmented_text_list))
        # N*N으로 증가시키기 위한 list
        temp_aug = augmented_text_list
    return augmented_text_list
            

In [28]:
bio_list = augmentation_bio_pattern(pattern, dict_entity)
bio_list

['B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ',
 'B_date 날짜 B_loc B_travel 정보   ']

## NER을 위한 Full Train Text 확보

In [29]:
ner_train_text = [augmented_text_list, bio_list]
ner_train_text

[['오늘 날짜 야탑 관광 정보   ',
  '내일 날짜 야탑 관광 정보   ',
  '모레 날짜 야탑 관광 정보   ',
  '오늘 날짜 판교 관광 정보   ',
  '내일 날짜 판교 관광 정보   ',
  '모레 날짜 판교 관광 정보   ',
  '오늘 날짜 야탑 여행 정보   ',
  '내일 날짜 야탑 여행 정보   ',
  '모레 날짜 야탑 여행 정보   ',
  '오늘 날짜 판교 여행 정보   ',
  '내일 날짜 판교 여행 정보   ',
  '모레 날짜 판교 여행 정보   ',
  '오늘 날짜 야탑 카페 정보   ',
  '내일 날짜 야탑 카페 정보   ',
  '모레 날짜 야탑 카페 정보   ',
  '오늘 날짜 판교 카페 정보   ',
  '내일 날짜 판교 카페 정보   ',
  '모레 날짜 판교 카페 정보   '],
 ['B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date 날짜 B_loc B_travel 정보   ',
  'B_date